In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance
from xgboost import cv

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics

os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [ ]:
'''mean-std noralization'''
def normalization_1(data):
    
    data_mean = np.mean(data,0)   #(72,)
    data_std = np.std(data,0,ddof=1) 
    data_ = (data - data_mean)/data_std
    
    return data_, data_mean , data_std

In [ ]:
'''max-min noralization'''
def normalization_2(data):
    
    data_min = np.min(data, 0)
    data_max = np.max(data, 0) 
    data_ = (data - data_min) / (data_max - data_min + 1e-7)
    
    return data_, data_min , data_max

In [ ]:
'''one-hot label'''
def to_categorical(y, num_classes=None):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [ ]:
'''one-hot label'''
# first arg: label
# second arg: num of class 
def one_hot ( labels , Label_class ): 
    one_hot_label = np.array([[ int (i == int (labels[j])) for i in range (Label_class)] for j in range ( len (labels))])      
    return one_hot_label

In [ ]:
'''doing wavelet on input data, and spilt them by slidding window'''
def add_window(x, time_step):
    
    x_window = []

    for i in range(x.shape[0]):
        series = x[i]
        series_window = []
        for j in range(int(series.shape[0]//time_step)):
            dat = series[j: j+time_step]
            series_window.append(dat.reshape([-1]))
        x_window.append(series_window)
    
    return np.array(x_window)

In [ ]:
'''spilt labe by slidding window in order to be align with input data'''

def label_add_window(x, time_step):
    
    x_window = []

    for i in range(x.shape[0]):
        series = x[i]
        series_window = []
        for j in range(int(series.shape[0]//time_step)):
            dat = series[j: j+time_step]
            series_window.append(dat[0])
        x_window.append(series_window)
    
    return np.array(x_window)

# Training set

In [ ]:
filedir = "tep_train"
filenames = []
train_feature = []
train_lable = []

for filename in os.listdir(filedir):
    filenames.append(os.path.join(filedir,filename))

# print(filenames)
    
filenames = [
'tep_train\\d01.dat', 'tep_train\\d02.dat', 'tep_train\\d03.dat',
'tep_train\\d04.dat', 'tep_train\\d05.dat', 'tep_train\\d06.dat',
'tep_train\\d07.dat', 'tep_train\\d08.dat',
'tep_train\\d10.dat', 'tep_train\\d11.dat', 'tep_train\\d12.dat', 
'tep_train\\d13.dat', 'tep_train\\d14.dat', 'tep_train\\d15.dat',
'tep_train\\d16.dat', 'tep_train\\d17.dat', 'tep_train\\d18.dat',
'tep_train\\d19.dat', 'tep_train\\d20.dat', 'tep_train\\d21.dat'
]
print("num of class, aka the amount of abnormality: ",len(filenames))


for i in range(len(filenames)):
    tep = np.genfromtxt(filenames[i])
    train_feature.append(tep)
    
    label = np.ones(tep.shape[0])*i
    train_lable.append(label)
    
train_feature = np.array(train_feature)
train_lable = np.array(train_lable)

print(train_feature.shape)
print(train_lable.shape)

# Test set


In [ ]:
filedir = "tep_test"
filenames_test = []

test_normal = []
test_fault = []
val = []

test_normal_label = []
test_fault_label = []
val_label = []

for filename in os.listdir(filedir):
    filenames_test.append(os.path.join(filedir,filename))
    
# print(filenames_test)

filenames_test = [
'tep_test\\d01_te.dat', 'tep_test\\d02_te.dat', 'tep_test\\d03_te.dat',
'tep_test\\d04_te.dat', 'tep_test\\d05_te.dat', 'tep_test\\d06_te.dat',
'tep_test\\d07_te.dat', 'tep_test\\d08_te.dat',
'tep_test\\d10_te.dat', 'tep_test\\d11_te.dat', 'tep_test\\d12_te.dat', 
'tep_test\\d13_te.dat', 'tep_test\\d14_te.dat', 'tep_test\\d15_te.dat', 
'tep_test\\d16_te.dat', 'tep_test\\d17_te.dat', 'tep_test\\d18_te.dat', 
'tep_test\\d19_te.dat', 'tep_test\\d20_te.dat', 'tep_test\\d21_te.dat'
]
    
print("num of class, aka the amount of abnormality: ",len(filenames_test))


for i in range(len(filenames_test)):
    
    tep = np.genfromtxt(filenames_test[i])
    test_normal_ = tep[:160]
    test_normal.append(test_normal_)

    val_ = tep[160:160+300]
    val.append(val_)
    
    test_fault_ = tep[160+300:]
    test_fault.append(test_fault_)

    label = np.ones(tep.shape[0])*i
    test_normal_label.append(label[:160])
    test_fault_label.append(label[160+300:])
    val_label.append(label[160:160+300])
    
test_normal = np.array(test_normal)
test_fault = np.array(test_fault)
val = np.array(val)
test_normal_label = np.array(test_normal_label)
test_fault_label = np.array(test_fault_label)
val_label = np.array(val_label)

print(test_normal.shape)
print(test_fault.shape)
print(val.shape)
print(test_normal_label.shape)
print(test_fault_label.shape)
print(val_label.shape)

In [ ]:
time_step = 1

train_x = add_window(train_feature, time_step).reshape([-1, time_step*tep.shape[-1]])
train_x, data_mean , data_std = normalization_1(train_x)
# train_x, data_min , data_max = normalization_2(train_x)
train_y = label_add_window(train_lable, time_step).reshape([-1])

val_x = add_window(val, time_step).reshape([-1, time_step*tep.shape[-1]])
val_x = (val_x - data_mean)/data_std
# val_x = (val_x - data_min) / (data_max - data_min + 1e-7)
val_y = label_add_window(val_label, time_step).reshape([-1])

test_normal_x = add_window(test_normal, time_step).reshape([-1, time_step*tep.shape[-1]])
test_normal_x = (test_normal_x - data_mean)/data_std
# test_normal_x = (test_normal_x - data_min) / (data_max - data_min + 1e-7)
test_normal_y = label_add_window(test_normal_label, time_step).reshape([-1])

test_fault_x = add_window(test_fault, time_step).reshape([-1, time_step*tep.shape[-1]])
test_fault_x = (test_fault_x - data_mean)/data_std
# test_fault_x = (test_fault_x - data_min) / (data_max - data_min + 1e-7)
test_fault_y = label_add_window(test_fault_label, time_step).reshape([-1])


print('train_x: ',train_x.shape)
print('train_y: ',train_y.shape)
print('----------------------------------------------')
print('val_x: ',val_x.shape)
print('val_y: ',val_y.shape)
print('----------------------------------------------')
print('test_normal_x: ',test_normal_x.shape)
print('test_normal_y: ',test_normal_y.shape)
print('----------------------------------------------')
print('test_fault_x: ',test_fault_x.shape)
print('test_fault_y: ',test_fault_y.shape)

In [ ]:
dtrain = xgb.DMatrix(train_x, train_y)
dval = xgb.DMatrix(val_x, val_y)
dtest_normal = xgb.DMatrix(test_normal_x, test_normal_y)
dtest_fault = xgb.DMatrix(test_fault_x, test_fault_y)

In [ ]:
params = {
'objective': 'multi:softprob',
'num_class': len(filenames),
'seed': 0,
'gamma': 0,
'max_depth': 10, #20
# 'random_state': 0,
'subsample': 0.3,
'min_child_weight': 0.5,
'lambda': 3.5,
'grow_policy': 'lossguide',
'eta': 0.007,
'eval_metric': ['merror'],
}

model = xgb.train(params, dtrain, 
          num_boost_round = 2000, 
          verbose_eval = 20, 
          early_stopping_rounds = 200, 
          evals=[(dtrain, 'train') , (dval, 'valid'), (dtest_normal, 'test_normal'), (dtest_fault, 'test_fault')],
          )

print('training finish')

In [ ]:
y_pred = model.predict(xgb.DMatrix(val_x))
yprob = np.argmax(y_pred, axis=1)  # return the index of the biggest pro

predictions = [round(value) for value in yprob]

# evaluate predictions
accuracy = accuracy_score(val_y, predictions)
print("Accuracy: %.5f%%" % (accuracy * 100.0))
print('Recall: %.4f' % metrics.recall_score(val_y, predictions, average='macro'))
print('F1-score: %.4f' % metrics.f1_score(val_y, predictions, average='macro'))
print('Precesion: %.4f' % metrics.precision_score(val_y, predictions, average='macro'))
print("confusion_matrix:")
print(confusion_matrix(val_y, predictions))
# print("%2.5f" %(confusion_matrix(val_y, predictions)))

In [ ]:
df = pd.DataFrame(confusion_matrix(val_y, predictions))
df.to_csv('./TEP_xgboost_val.csv')

In [ ]:
y_pred = model.predict(xgb.DMatrix(test_fault_x))
yprob = np.argmax(y_pred, axis=1)  # return the index of the biggest pro

predictions = [round(value) for value in yprob]

# evaluate predictions
accuracy = accuracy_score(test_fault_y, predictions)
print("Accuracy: %.5f%%" % (accuracy * 100.0))
print('Recall: %.4f' % metrics.recall_score(test_fault_y, predictions, average='macro'))
print('F1-score: %.4f' % metrics.f1_score(test_fault_y, predictions, average='macro'))
print('Precesion: %.4f' % metrics.precision_score(test_fault_y, predictions, average='macro'))
print("confusion_matrix:")
print(confusion_matrix(test_fault_y, predictions))
df = pd.DataFrame(confusion_matrix(test_fault_y, predictions))
df.to_csv('./TEP_xgboost_test.csv')

In [ ]:
def calculate_all_prediction(confMatrix):

    total_sum=confMatrix.sum()
    correct_sum=(np.diag(confMatrix)).sum()
    prediction=round(100*float(correct_sum)/float(total_sum),2)
    print('acc:'+str(prediction)+'%')
    
calculate_all_prediction(confusion_matrix(test_fault_y, predictions))

In [ ]:
classes=['1','2','3','4','5','6','7','8','10','11','12','13','14','15','16','17','18','19','20','21']


def calculae_lable_prediction(confMatrix):
    l=len(confMatrix)
    for i in range(l):
        label_total_sum = confMatrix.sum(axis=1)[i]
        label_correct_sum=confMatrix[i][i]
        prediction = round(100 * float(label_correct_sum) / float(label_total_sum), 2)
        print('prediction:'+classes[i]+":"+str(prediction)+'%')
        
calculae_lable_prediction(confusion_matrix(test_fault_y, predictions))

In [ ]:

def calculate_label_recall(confMatrix):
    l = len(confMatrix)
    for i in range(l):
        label_total_sum = confMatrix.sum(axis=0)[i]
        label_correct_sum = confMatrix[i][i]
        prediction = round(100 * float(label_correct_sum) / float(label_total_sum), 2)
        print('recall:'+classes[i] + ":" + str(prediction) + '%')

calculate_label_recall(confusion_matrix(test_fault_y, predictions))